## Setup

In [1]:
# carregamento das bibliotecas
import sys
import os
import pandas as pd 
from datetime import date, timedelta
# Adiciona a pasta raiz do projeto (onde está a pasta src) ao sys.path
sys.path.append(os.path.abspath(os.path.join("..")))
from src.config.logging_config import setup_logging
from src.data.load_data import carregar_dataset
from src.etl.webscrapping import gerar_tabela_cotacao_diaria_ibovespa
from src.data.preprocessing import *

In [2]:
# configuração do logging
setup_logging()


## Carregamento dos dados

In [3]:
# carregamento da tabela de ações componentes do ibovespa com dados  setorais
tbl_carteira_ibovespa = carregar_dataset(path='../data/external/tbl_acoes_ibovespa.csv')
tbl_carteira_ibovespa

2025-08-05 19:40:20,161 | INFO | src.data.load_data | Captura do arquivo csv no path:../data/external/tbl_acoes_ibovespa.csv


,codigo,tipo,empresa,slug,setor,industria,ticker
0,ABEV3,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ABEV3.SA
1,ALOS3,ON,Allos,alos3,Imóveis,Atividades Imobiliárias,ALOS3.SA
2,ASAI3,ON,Assaí­ Atacadista,assai-asai3,Consumo não-cíclico,Varejo de alimentos e medicamentos,ASAI3.SA
3,AURE3,ON,Auren,auren-energia-aure3,Utilidades,Empresas independentes e concessionárias elétr...,AURE3.SA
4,AZZA3,ON,Azzas 2154,azzas-2154-azza3,Consumo cíclico,Varejistas especializados,AZZA3.SA
...,...,...,...,...,...,...,...
79,CPLE6,PNB,Companhia Paranaense de Energia COPEL,copel-pnb,Utilidades,Empresas independentes e concessionárias elétr...,CPLE6.SA
80,TIMS3,ON,TIM SA,tim-part-s-a-on-nm,Tecnologia,Serviços de telecomunicação,TIMS3.SA
81,VAMO3,ON,Grupo Vamos,grupo-vamos-sa,Indústria,Serviços de frete e logística,VAMO3.SA
82,VBBR3,ON,Vibra Energia SA,petrobras-distribuidora,Energia,Gás e Petróleo,VBBR3.SA


In [4]:
# carregamento da base historica de ações componentes do ibovespa para validação dos dados
tbl_cotacao_ibovespa = carregar_dataset(path='../data/raw/tbl_acoes_ibovespa_historico.csv')
tbl_cotacao_ibovespa.tail()

2025-08-05 19:40:23,272 | INFO | src.data.load_data | Captura do arquivo csv no path:../data/raw/tbl_acoes_ibovespa_historico.csv


,Date,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,returnOnEquity,profitMargins,revenueGrowth,dividendYield,totalDebt,freeCashflow
835,2025-07-21 00:00:00-03:00,VIVA3,25.580000,26.000000,25.290001,25.700001,4756700,0.0,0.0,8.243589,2.314615,0.31979,0.27443,0.208,2.6,1063061696,-336666272.0
836,2025-07-22 00:00:00-03:00,VIVA3,25.730000,25.879999,24.790001,24.790001,3152000,0.0,0.0,8.243589,2.314615,0.31979,0.27443,0.208,2.6,1063061696,-336666272.0
837,2025-07-23 00:00:00-03:00,VIVA3,24.700001,25.570000,24.420000,25.389999,1841600,0.0,0.0,8.243589,2.314615,0.31979,0.27443,0.208,2.6,1063061696,-336666272.0
838,2025-07-24 00:00:00-03:00,VIVA3,25.080000,25.350000,24.910000,25.049999,1806600,0.0,0.0,8.243589,2.314615,0.31979,0.27443,0.208,2.6,1063061696,-336666272.0
839,2025-07-25 00:00:00-03:00,VIVA3,24.930000,25.320000,24.930000,25.250000,1832200,0.0,0.0,8.243589,2.314615,0.31979,0.27443,0.208,2.6,1063061696,-336666272.0


In [ ]:
# nova iteração: ajuste dos dados da cotacao para gerar um historico maior para validação de tendências pelos modelos
hoje = date.today()
tbl_cotacao_ibovespa = gerar_tabela_cotacao_diaria_ibovespa(lista_acoes=tbl_carteira_ibovespa['ticker'], data_inicio=hoje-timedelta(120), data_cotacao=hoje-timedelta(12))
tbl_cotacao_ibovespa.head()

2025-08-05 19:39:57,567 | INFO | src.etl.webscrapping | Iniciando o processo de captação dos dados.
2025-08-05 19:40:00,342 | INFO | src.etl.webscrapping | Dados coletados para: ABEV3.SA
2025-08-05 19:40:01,338 | ERROR | yfinance | $ALOS3.SA: possibly delisted; no price data found  (1d 2025-04-07 -> 2025-07-24)


Exception ignored from cffi callback <function buffer_callback at 0x000002331CC5BEC0>:
Traceback (most recent call last):
  File "c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\.venv\Lib\site-packages\curl_cffi\curl.py", line 68, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


2025-08-05 19:40:02,180 | WARNING | src.etl.webscrapping | Nenhum dado retornado para a ação: ALOS3.SA após múltiplas tentativas.
2025-08-05 19:40:03,533 | INFO | src.etl.webscrapping | Dados coletados para: ASAI3.SA
2025-08-05 19:40:05,615 | INFO | src.etl.webscrapping | Dados coletados para: AURE3.SA
2025-08-05 19:40:07,746 | INFO | src.etl.webscrapping | Dados coletados para: AZZA3.SA
2025-08-05 19:40:10,060 | INFO | src.etl.webscrapping | Dados coletados para: B3SA3.SA
2025-08-05 19:40:12,142 | INFO | src.etl.webscrapping | Dados coletados para: BBAS3.SA


In [ ]:
# concatenação dos dados setoriais com a tabela de cotação
tbl_cotacao_ibovespa = tbl_cotacao_ibovespa.merge(tbl_carteira_ibovespa.drop(['ticker'], axis=1), left_on='ticker', right_on='codigo', how='left')
tbl_cotacao_ibovespa.drop(columns='codigo', axis=1, inplace=True)
tbl_cotacao_ibovespa.head()

,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,...,tipo_x,empresa_x,slug_x,setor_x,industria_x,tipo_y,empresa_y,slug_y,setor_y,industria_y
0,ABEV3,13.171949,13.479196,12.874613,13.062926,50069000,0.0,0.0,13.148936,2.083966,...,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
1,ABEV3,13.043103,13.360261,13.033192,13.122393,39798300,0.0,0.0,13.148936,2.083966,...,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
2,ABEV3,13.072837,13.756708,13.033192,13.548573,51912400,0.0,0.0,13.148936,2.083966,...,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
3,ABEV3,13.548574,13.617951,13.399906,13.479196,21741000,0.0,0.0,13.148936,2.083966,...,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
4,ABEV3,13.608040,13.608040,13.320616,13.360261,35635500,0.0,0.0,13.148936,2.083966,...,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas


: 

In [15]:
tbl_cotacao_ibovespa

,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,...,profitMargins,revenueGrowth,dividendYield,totalDebt,freeCashflow,tipo,empresa,slug,setor,industria
0,ABEV3,13.171949,13.479196,12.874613,13.062926,50069000,0.0,0.0,13.148936,2.083966,...,0.16084,0.002,8.47,3157764096,17938016256,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
1,ABEV3,13.043103,13.360261,13.033192,13.122393,39798300,0.0,0.0,13.148936,2.083966,...,0.16084,0.002,8.47,3157764096,17938016256,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
2,ABEV3,13.072837,13.756708,13.033192,13.548573,51912400,0.0,0.0,13.148936,2.083966,...,0.16084,0.002,8.47,3157764096,17938016256,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
3,ABEV3,13.548574,13.617951,13.399906,13.479196,21741000,0.0,0.0,13.148936,2.083966,...,0.16084,0.002,8.47,3157764096,17938016256,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
4,ABEV3,13.608040,13.608040,13.320616,13.360261,35635500,0.0,0.0,13.148936,2.083966,...,0.16084,0.002,8.47,3157764096,17938016256,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6073,VIVA3,25.309999,25.420000,24.969999,25.240000,1778200,0.0,0.0,8.657052,2.430706,...,0.27443,0.208,2.48,1063061696,-336666272,ON,Vivara Participações SA,vivara-participacoes-sa,Consumo cíclico,Varejistas especializados
6074,VIVA3,24.990000,25.830000,24.430000,25.580000,8006800,0.0,0.0,8.657052,2.430706,...,0.27443,0.208,2.48,1063061696,-336666272,ON,Vivara Participações SA,vivara-participacoes-sa,Consumo cíclico,Varejistas especializados
6075,VIVA3,25.580000,26.000000,25.290001,25.700001,4756700,0.0,0.0,8.657052,2.430706,...,0.27443,0.208,2.48,1063061696,-336666272,ON,Vivara Participações SA,vivara-participacoes-sa,Consumo cíclico,Varejistas especializados
6076,VIVA3,25.730000,25.879999,24.790001,24.790001,3152000,0.0,0.0,8.657052,2.430706,...,0.27443,0.208,2.48,1063061696,-336666272,ON,Vivara Participações SA,vivara-participacoes-sa,Consumo cíclico,Varejistas especializados


In [14]:
# transformação da coluna date em index do dataset
tbl_cotacao_ibovespa.set_index('Date', drop=True, inplace=True)
tbl_cotacao_ibovespa.index = pd.to_datetime(tbl_cotacao_ibovespa.index)

KeyError: "None of ['Date'] are in the columns"

# Gerar um novo historico para dados fundamentalistas para um novo processo de re-treino futuro do modelo
geram uma repetição dos dados com a execução da api, não recuperando o histórico

In [ ]:
tbl_cotacao_ibovespa.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6152 entries, 2025-04-07 00:00:00-03:00 to 2025-07-23 00:00:00-03:00
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ticker          6152 non-null   object 
 1   Open            6152 non-null   float64
 2   High            6152 non-null   float64
 3   Low             6152 non-null   float64
 4   Close           6152 non-null   float64
 5   Volume          6152 non-null   int64  
 6   Dividends       6152 non-null   float64
 7   Stock Splits    6152 non-null   float64
 8   trailingPE      5264 non-null   float64
 9   priceToBook     6152 non-null   float64
 10  returnOnEquity  6078 non-null   float64
 11  profitMargins   6152 non-null   float64
 12  revenueGrowth   6078 non-null   float64
 13  dividendYield   5466 non-null   float64
 14  totalDebt       6152 non-null   int64  
 15  freeCashflow    5708 non-null   object 
dtypes: float64(12), int64(2), obje

In [ ]:
tbl_cotacao_ibovespa.columns

Index(['ticker', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [ ]:
# nova iteração: remoção das colunas fundamentalistas,pois elas geram uma repetição dos dados com a execução da api, não recuperando o histórico
tbl_cotacao_ibovespa.drop(columns=['Dividends',
       'Stock Splits', 'trailingPE', 'priceToBook', 'returnOnEquity',
       'profitMargins', 'revenueGrowth', 'dividendYield', 'totalDebt',
       'freeCashflow'], axis=1, inplace=True)

In [ ]:
# inserção de dados temporais no dataset

tbl_cotacao_ibovespa = dados_temporais(df=tbl_cotacao_ibovespa)
tbl_cotacao_ibovespa.head()


,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,...,totalDebt,freeCashflow,tipo,empresa,slug,setor,industria,dayofweek,month,Feriado
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-14 00:00:00-03:00,ABEV3,13.26,13.57,13.26,13.29,36420400,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,0,7,False
2025-07-15 00:00:00-03:00,ABEV3,13.37,13.52,13.23,13.27,27675300,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,1,7,False
2025-07-16 00:00:00-03:00,ABEV3,13.38,13.73,13.31,13.68,48903600,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,2,7,False
2025-07-17 00:00:00-03:00,ABEV3,13.67,13.69,13.44,13.46,19213600,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,3,7,False
2025-07-18 00:00:00-03:00,ABEV3,13.33,13.49,13.29,13.32,23902900,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,4,7,False


In [ ]:
# transformação da colunas Feriado no formato int
tbl_cotacao_ibovespa['Feriado'] = tbl_cotacao_ibovespa['Feriado'].astype(int)

P/L (Preço sobre Lucro) (trailingPE): Quanto o mercado paga por cada real de lucro da empresa. Ajuda a ver se a ação está cara ou barata em relação ao lucro.

P/VP (Preço sobre Valor Patrimonial) (priceToBook): Compara o preço da ação com o valor contábil dos ativos da empresa. Indica se a ação está acima ou abaixo do seu valor patrimonial.

ROE (Retorno sobre o Patrimônio Líquido) (returnOnEquity): Rentabilidade da empresa sobre o capital próprio investido pelos acionistas. Um ROE alto mostra boa gestão.

Margem Líquida (profitMargins): Porcentagem da receita que se transforma em lucro líquido. Indica a eficiência da empresa em gerar lucro das vendas.

Crescimento de Receita (revenueGrowth): Aumento percentual das vendas da empresa. Essencial para avaliar a expansão e potencial de mercado.

Dividend Yield (dividendYield): Retorno em dividendos que a ação oferece em relação ao seu preço. Importante para quem busca renda passiva.

Dívida Total (totalDebt): Montante total de obrigações financeiras da empresa. Ajuda a avaliar a saúde financeira e o risco.

Fluxo de Caixa Livre (freeCashflow): Dinheiro que sobra para a empresa após todas as despesas e investimentos. Sinaliza a capacidade da empresa de gerar caixa puro.

## Análise

In [ ]:
# informações da tabela
tbl_cotacao_ibovespa.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 840 entries, 2025-07-14 00:00:00-03:00 to 2025-07-25 00:00:00-03:00
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ticker          840 non-null    object 
 1   Open            840 non-null    float64
 2   High            840 non-null    float64
 3   Low             840 non-null    float64
 4   Close           840 non-null    float64
 5   Volume          840 non-null    int64  
 6   Dividends       840 non-null    float64
 7   Stock Splits    840 non-null    float64
 8   trailingPE      720 non-null    float64
 9   priceToBook     840 non-null    float64
 10  returnOnEquity  830 non-null    float64
 11  profitMargins   840 non-null    float64
 12  revenueGrowth   830 non-null    float64
 13  dividendYield   740 non-null    float64
 14  totalDebt       840 non-null    int64  
 15  freeCashflow    780 non-null    float64
 16  tipo            840 non-null   

In [ ]:
# verificação da quantidade de ações por industria
empresas_por_industria = agrupar_dados(df=tbl_cotacao_ibovespa, cols_agrup=['setor', 'industria'], cols_filter=['ticker'], agr='nunique').reset_index()
empresas_por_industria.head()

2025-08-05 15:00:40,629 | INFO | src.data.preprocessing | Agrupamento selecionado: ['setor', 'industria'], filtragem dataset:['ticker'], método: nunique


,setor,industria,ticker
0,Saúde,Prestação de serviços de Saúde,1
1,Serviços acadêmicos e educacionais,"Colégio, faculdade e universidade",1
2,Consumo cíclico,Serviços de hotel e entretenimento,2
3,Consumo cíclico,Automóveis e peças,1
4,Consumo cíclico,Varejistas especializados,4


In [ ]:
industrias_por_setor = tbl_cotacao_ibovespa.groupby('setor')['industria'].nunique().reset_index(name='count_industria_setor')
industrias_por_setor

,setor,count_industria_setor
0,Saúde,1
1,Serviços acadêmicos e educacionais,1
2,Consumo cíclico,4
3,Consumo não-cíclico,6
4,Energia,1
5,Financeiro,3
6,Imóveis,2
7,Indústria,5
8,Materiais básicos,4
9,Saúde,2


In [ ]:
# verificação da quantidade de ações por setor
empresas_por_setor = tbl_cotacao_ibovespa.groupby(['setor'])['ticker'].nunique().reset_index(name='count_ticker_setor')
empresas_por_setor.head()

,setor,count_ticker_setor
0,Saúde,1
1,Serviços acadêmicos e educacionais,1
2,Consumo cíclico,8
3,Consumo não-cíclico,11
4,Energia,8


In [ ]:
# verificação de nulos
tbl_cotacao_ibovespa.isna().sum()

ticker              0
Open                0
High                0
Low                 0
Close               0
Volume              0
Dividends           0
Stock Splits        0
trailingPE        120
priceToBook         0
returnOnEquity     10
profitMargins       0
revenueGrowth      10
dividendYield     100
totalDebt           0
freeCashflow       60
tipo                0
empresa             0
slug                0
setor               0
industria           0
dayofweek           0
month               0
Feriado             0
dtype: int64

In [ ]:
#valores_nulos = tbl_cotacao_ibovespa[tbl_cotacao_ibovespa.isnull().any(axis=1)][['ticker', 'trailingPE', 'dividendYield', 'freeCashflow']]
#valores_nulos.head()

,ticker,trailingPE,dividendYield,freeCashflow
Date,,,,
2025-07-14 00:00:00-03:00,BBAS3,4.816273,1.84,NaN
2025-07-15 00:00:00-03:00,BBAS3,4.816273,1.84,NaN
2025-07-16 00:00:00-03:00,BBAS3,4.816273,1.84,NaN
2025-07-17 00:00:00-03:00,BBAS3,4.816273,1.84,NaN
2025-07-18 00:00:00-03:00,BBAS3,4.816273,1.84,NaN


# Prenchimento de nulos
Para novos dados no pipeline, usaremos o método de preenchimento ffill ou com a média movel até o dia anterior para preenchimento dos valores nulos no glue.

In [ ]:
# tratamento dos valores nulos: interpolação em cascata
#tbl_cotacao_ibovespa = tratamento_nulo_dados_setor_industria(df=tbl_cotacao_ibovespa, colunas=['trailingPE','returnOnEquity','revenueGrowth','dividendYield','freeCashflow'])
#tbl_cotacao_ibovespa.head()

2025-08-05 15:00:40,739 | INFO | src.data.preprocessing | Iniciando a preparação do dataset para interpolação.
2025-08-05 15:00:40,756 | INFO | src.data.preprocessing | Contagens por setor e indústria adicionadas ao DataFrame.
2025-08-05 15:00:40,757 | INFO | src.data.preprocessing | Processando coluna: trailingPE
2025-08-05 15:00:40,792 | INFO | src.data.preprocessing | Processando coluna: returnOnEquity
2025-08-05 15:00:40,808 | INFO | src.data.preprocessing | Processando coluna: revenueGrowth
2025-08-05 15:00:40,821 | INFO | src.data.preprocessing | Processando coluna: dividendYield
2025-08-05 15:00:40,833 | INFO | src.data.preprocessing | Processando coluna: freeCashflow
2025-08-05 15:00:40,844 | INFO | src.data.preprocessing | Aplicando ffill() e bfill() finais.
2025-08-05 15:00:40,847 | WARNING | src.data.preprocessing | Ainda há NaNs em 'trailingPE'. Preenchendo com a média global.
2025-08-05 15:00:40,850 | WARNING | src.data.preprocessing | Ainda há NaNs em 'returnOnEquity'. Pr

c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\src\data\preprocessing.py:270: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transf[coluna].fillna(media, inplace=True)


,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,...,totalDebt,freeCashflow,tipo,empresa,slug,setor,industria,dayofweek,month,Feriado
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-14 00:00:00-03:00,ABEV3,13.26,13.57,13.26,13.29,36420400,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,0,7,0
2025-07-15 00:00:00-03:00,ABEV3,13.37,13.52,13.23,13.27,27675300,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,1,7,0
2025-07-16 00:00:00-03:00,ABEV3,13.38,13.73,13.31,13.68,48903600,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,2,7,0
2025-07-17 00:00:00-03:00,ABEV3,13.67,13.69,13.44,13.46,19213600,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,3,7,0
2025-07-18 00:00:00-03:00,ABEV3,13.33,13.49,13.29,13.32,23902900,0.0,0.0,13.215054,2.034432,...,3218170112,1.830223e+10,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,4,7,0


In [ ]:
# verificação de duplicados
tbl_carteira_ibovespa.duplicated().sum()

0

In [ ]:
# análise descritivos interativa dos dados
boxplot_analise_descritiva_categorica(tbl_cotacao_ibovespa,distribuicao=(0.1,0.25,0.5,0.75,0.95,0.99), feature='ticker')

interactive(children=(Dropdown(description='coluna', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# histograma do target

In [ ]:
# histograma do target com escala
histograma_feature_categorica(tbl_cotacao_ibovespa[['ticker','Close']],feature='ticker')

interactive(children=(Dropdown(description='coluna', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

# Feature Engineering

In [ ]:
# transformação cíclica para dias uteis 
tbl_cotacao_ibovespa = transformacao_ciclica(df=tbl_cotacao_ibovespa, dias_uteis=True)

2025-08-05 15:00:41,371 | INFO | src.data.preprocessing | Transformação cíclica com dias úteis para as colunas de dados temporais.


# análise de estacionariedade da serie para definição de lags e médias móveis

In [ ]:
# teste de estacionariedade
testar_estacionariedade_interativo(df= tbl_cotacao_ibovespa, coluna_valor = 'Close')

interactive(children=(Dropdown(description='ticker', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# gerar diferenciação do target para revalidar a estacionariedade da serie
tbl_cotacao_ibovespa = diferenciar_serie_temporal(df= tbl_cotacao_ibovespa, target = 'Close')


In [ ]:
# teste de estacionariedade com diferenciacao do target
testar_estacionariedade_interativo(df= tbl_cotacao_ibovespa, coluna_valor = 'Close_diff')

interactive(children=(Dropdown(description='ticker', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# decomposicao da serie temporal diferenciada
grafico_decomposicao_temporal_interativo(df= tbl_cotacao_ibovespa, target = 'Close', period=5)

interactive(children=(Dropdown(description='ticker', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# gráfico de autocorrelacao para definição do número de lags e médias móveis
grafico_acf_interativo(df=tbl_cotacao_ibovespa,coluna_valor = 'Close_diff',max_lags=8)

interactive(children=(Dropdown(description='ticker', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# gráfico de autocorrelacao parcial para definição do número de lags e médias móveis
grafico_pacf_interativo(df=tbl_cotacao_ibovespa,coluna_valor = 'Close_diff',max_lags=9)

interactive(children=(Dropdown(description='ticker', options=('ABEV3', 'ALOS3', 'ASAI3', 'AURE3', 'AZZA3', 'B3…

In [ ]:
# inserção das colunas de janela para o target com e sem diferenciacao
tbl_cotacao_ibovespa = gerar_features_temporais(df=tbl_cotacao_ibovespa,coluna_valor='Close_diff')
tbl_cotacao_ibovespa[['day_sin', 'day_cos', 'month_sin','month_cos']].head(20)

,day_sin,day_cos,month_sin,month_cos
Date,,,,
2025-07-15 00:00:00-03:00,0.951057,0.309017,-0.5,-0.866025
2025-07-16 00:00:00-03:00,0.587785,-0.809017,-0.5,-0.866025
2025-07-17 00:00:00-03:00,-0.587785,-0.809017,-0.5,-0.866025
2025-07-18 00:00:00-03:00,-0.951057,0.309017,-0.5,-0.866025
2025-07-21 00:00:00-03:00,0.000000,1.000000,-0.5,-0.866025
2025-07-22 00:00:00-03:00,0.951057,0.309017,-0.5,-0.866025
2025-07-23 00:00:00-03:00,0.587785,-0.809017,-0.5,-0.866025
2025-07-24 00:00:00-03:00,-0.587785,-0.809017,-0.5,-0.866025
2025-07-25 00:00:00-03:00,-0.951057,0.309017,-0.5,-0.866025


In [ ]:
# Considerações sobre colunas criadas para teste de modelo

In [ ]:
tbl_cotacao_ibovespa.columns

Index(['ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'trailingPE', 'priceToBook', 'returnOnEquity',
       'profitMargins', 'revenueGrowth', 'dividendYield', 'totalDebt',
       'freeCashflow', 'tipo', 'empresa', 'slug', 'setor', 'industria',
       'dayofweek', 'month', 'Feriado', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'Close_diff', 'lag_1_Close', 'lag_2_Close', 'lag_3_Close',
       'lag_5_Close', 'rolling_mean_3_Close', 'volatility_3_Close',
       'retorno_acumulado_3_Close', 'rolling_mean_5_Close',
       'volatility_5_Close', 'retorno_acumulado_5_Close', 'lag_1_Close_diff',
       'lag_2_Close_diff', 'lag_3_Close_diff', 'lag_5_Close_diff',
       'rolling_mean_3_Close_diff', 'volatility_3_Close_diff',
       'retorno_acumulado_3_Close_diff', 'rolling_mean_5_Close_diff',
       'volatility_5_Close_diff', 'retorno_acumulado_5_Close_diff'],
      dtype='object')

In [ ]:
# remoção de colunas para gerar dataset para validação de modelos
tbl_cotacao_ibovespa.drop(columns=['empresa', 'slug'], axis=1, inplace=True)

In [7]:
tbl_cotacao_ibovespa.columns

NameError: name 'tbl_cotacao_ibovespa' is not defined

In [ ]:
# renome das colunas para treinamento do modelo
tbl_cotacao_ibovespa.rename({"ticker":"acao", "Open":"abertura","High":"maximo", "Low":"minimo","Close":"fechamento","Volume":"volume"})


    df = df.withColumnRenamed("ticker", "acao")
    df = df.withColumnRenamed("data_cotacao", "date")
    df = df.withColumnRenamed("Open", "abertura")
    df = df.withColumnRenamed("High", "maximo")
    df = df.withColumnRenamed("Low", "minimo")
    df = df.withColumnRenamed("Close", "fechamento")
    df = df.withColumnRenamed("Volume", "volume")
    df = df.withColumnRenamed("Dividends", "dividendos")
    df = df.withColumnRenamed("Stock Splits", "desdobramentos")
    df = df.withColumnRenamed("trailingPE", "trailing_pe")
    df = df.withColumnRenamed("priceToBook", "price_to_book")
    df = df.withColumnRenamed("returnOnEquity", "return_on_equity")
    df = df.withColumnRenamed("profitMargins", "profit_margins")
    df = df.withColumnRenamed("revenueGrowth", "revenue_growth")
    df = df.withColumnRenamed("dividendYield", "dividend_yield")
    df = df.withColumnRenamed("totalDebt", "total_debt")
    df = df.withColumnRenamed("freeCashflow", "free_cashflow")

In [182]:
# salvar o dataset para treino e validação dos modelos
tbl_cotacao_ibovespa.to_csv('../data/processed/tbl_cotacao_ibovespa_processed.csv', index=True)

In [4]:
from src.etl.webscrapping import gerar_tabela_cotacao_diaria_ibovespa
from datetime import date, timedelta
hoje = date.today()
tbl_carteira_ibovespa = carregar_dataset(path='../data/external/tbl_acoes_ibovespa.csv')


2025-08-05 17:27:26,926 | INFO | src.data.load_data | Captura do arquivo csv no path:../data/external/tbl_acoes_ibovespa.csv


In [5]:
tbl = gerar_tabela_cotacao_diaria_ibovespa(lista_acoes=tbl_carteira_ibovespa['ticker'], data_inicio=hoje-timedelta(8), data_cotacao=hoje-timedelta(7))

2025-08-05 17:27:28,097 | INFO | src.etl.webscrapping | Iniciando o processo de captação dos dados.
2025-08-05 17:27:30,417 | INFO | src.etl.webscrapping | Dados coletados para: ABEV3.SA
2025-08-05 17:27:32,574 | INFO | src.etl.webscrapping | Dados coletados para: ALOS3.SA
2025-08-05 17:27:34,405 | INFO | src.etl.webscrapping | Dados coletados para: ASAI3.SA
2025-08-05 17:27:36,267 | INFO | src.etl.webscrapping | Dados coletados para: AURE3.SA
2025-08-05 17:27:38,152 | INFO | src.etl.webscrapping | Dados coletados para: AZZA3.SA
2025-08-05 17:27:40,267 | INFO | src.etl.webscrapping | Dados coletados para: B3SA3.SA
2025-08-05 17:27:42,377 | INFO | src.etl.webscrapping | Dados coletados para: BBAS3.SA
2025-08-05 17:27:44,255 | INFO | src.etl.webscrapping | Dados coletados para: BBDC3.SA
2025-08-05 17:27:46,128 | INFO | src.etl.webscrapping | Dados coletados para: BBDC4.SA
2025-08-05 17:27:47,987 | INFO | src.etl.webscrapping | Dados coletados para: BBSE3.SA
2025-08-05 17:27:50,114 | INFO

c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\src\etl\webscrapping.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tbl_cotacao_diaria = pd.concat(lista_df)


In [6]:
tbl

,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,returnOnEquity,profitMargins,revenueGrowth,dividendYield,totalDebt,freeCashflow
Date,,,,,,,,,,,,,,,,
2025-07-28 00:00:00-03:00,ABEV3,13.470000,13.490000,13.030000,13.060000,56671100,0.000000,0.0,13.148936,2.083966,0.16049,0.16084,0.002,8.47,3157764096,17938016256
2025-07-28 00:00:00-03:00,ALOS3,21.260000,21.480000,20.930000,21.049999,3215300,0.000000,0.0,13.409938,0.804846,0.06593,0.30534,0.007,5.57,6455017984,855961344
2025-07-28 00:00:00-03:00,ASAI3,9.700000,9.750000,9.480000,9.530000,14134200,0.000000,0.0,15.754097,2.406712,0.16384,0.01099,0.077,1.09,25693999104,-144307008
2025-07-28 00:00:00-03:00,AURE3,9.550000,9.660000,9.320000,9.320000,1868300,0.000000,0.0,159.333340,0.753884,0.00539,0.00060,1.113,0.59,26161764352,504947872
2025-07-28 00:00:00-03:00,AZZA3,36.560001,36.849998,34.919998,35.160000,684500,0.000000,0.0,14.301205,0.942088,0.07151,0.03863,1.515,6.24,4073274880,-1987710976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-28 00:00:00-03:00,CPLE6,11.670000,11.740000,11.420000,11.580000,11522800,0.000000,0.0,14.936708,1.333635,0.09577,0.12729,0.088,5.11,19138037760,-4095745536
2025-07-28 00:00:00-03:00,TIMS3,19.879999,19.990000,19.670000,19.799999,4567300,0.132315,0.0,14.369127,2.062024,0.14331,0.13922,0.047,6.12,16222999552,6694933504
2025-07-28 00:00:00-03:00,VAMO3,3.990000,4.020000,3.900000,3.910000,10950800,0.000000,0.0,6.627119,1.678832,0.17202,0.06173,0.236,15.22,16508325888,2585804288


In [4]:
import pandas as pd
df = pd.read_parquet('../data/external/cotacao_ibovespa.parquet')
df

,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,returnOnEquity,profitMargins,revenueGrowth,dividendYield,totalDebt,freeCashflow
Date,,,,,,,,,,,,,,,,
2025-08-04 00:00:00-03:00,ABEV3,12.390000,12.570000,12.340000,12.410000,16396200,0.0,0.0,13.234042,2.097454,0.16049,0.16084,0.002,8.47,3157764096,1.793802e+10
2025-08-04 00:00:00-03:00,ALOS3,21.879999,21.910000,21.430000,21.540001,2601500,0.0,0.0,13.347826,0.801118,0.06593,0.30534,0.007,5.57,6455017984,8.559613e+08
2025-08-04 00:00:00-03:00,ASAI3,9.800000,9.960000,9.670000,9.860000,14030300,0.0,0.0,15.901639,2.429251,0.16384,0.01099,0.077,1.09,25693999104,-1.443070e+08
2025-08-04 00:00:00-03:00,AURE3,9.640000,9.800000,9.630000,9.730000,1686800,0.0,0.0,160.000020,0.757038,0.00539,0.00060,1.113,0.59,26161764352,5.049479e+08
2025-08-04 00:00:00-03:00,AZZA3,35.830002,35.910000,34.779999,35.250000,1235200,0.0,0.0,14.144579,0.931771,0.07151,0.03863,1.515,6.24,4073274880,-1.987711e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04 00:00:00-03:00,CPLE6,12.030000,12.070000,11.780000,11.900000,7250500,0.0,0.0,14.898734,1.330244,0.09577,0.12729,0.088,5.11,19138037760,-4.095746e+09
2025-08-04 00:00:00-03:00,TIMS3,20.950001,21.350000,20.930000,21.350000,7032300,0.0,0.0,14.436242,2.071655,0.14331,0.13922,0.047,6.12,16222999552,6.694934e+09
2025-08-04 00:00:00-03:00,VAMO3,3.930000,3.960000,3.840000,3.860000,5698300,0.0,0.0,6.610170,1.674538,0.17202,0.06173,0.236,15.22,16508325888,2.585804e+09
